### Import necessary libraries

In [1]:
import pandas as pd
import pickle
import numpy as np
# from sklearn.model_selection import lo

### Load Dataset

In [2]:
df_val = pd.read_csv('../notebooks/datasets/test_data.csv', delimiter="\t", encoding='latin1', header=None)
df_excel = pd.read_excel('../notebooks/datasets/description.xls', index_col=0, engine="xlrd")

df_val.head()

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,70001,C,10,Web,NaN,1,F,1,0,5,...,NaN,0,0,0,0,2,N,29,318,318
1,70002,C,10,Web,NaN,1,F,1,0,3,...,2.0,0,0,0,0,2,N,41,173,173
2,70003,C,25,Web,NaN,1,F,2,0,4,...,NaN,0,0,0,0,1,N,38,716,716
3,70004,C,5,Web,NaN,1,F,1,0,3,...,NaN,0,0,0,0,1,N,45,869,869
4,70005,C,20,Web,NaN,1,F,1,0,4,...,NaN,0,0,0,0,1,N,49,289,289


### Add columns title to dataset

In [3]:
metadata = df_excel

meta_cols = metadata["Var_Title"].to_list()[:-1]
meta_cols[43] = "MATE_EDUCATION_LEVEL"

# Set the new column to the train_data and test_data
df_val.columns = meta_cols
df_val.columns


Index(['ID_CLIENT', 'CLERK_TYPE', 'PAYMENT_DAY', 'APPLICATION_SUBMISSION_TYPE',
       'QUANT_ADDITIONAL_CARDS', 'POSTAL_ADDRESS_TYPE', 'SEX',
       'MARITAL_STATUS', 'QUANT_DEPENDANTS', 'EDUCATION_LEVEL',
       'STATE_OF_BIRTH', 'CITY_OF_BIRTH', 'NACIONALITY', 'RESIDENCIAL_STATE',
       'RESIDENCIAL_CITY', 'RESIDENCIAL_BOROUGH', 'FLAG_RESIDENCIAL_PHONE',
       'RESIDENCIAL_PHONE_AREA_CODE', 'RESIDENCE_TYPE', 'MONTHS_IN_RESIDENCE',
       'FLAG_MOBILE_PHONE', 'FLAG_EMAIL', 'PERSONAL_MONTHLY_INCOME',
       'OTHER_INCOMES', 'FLAG_VISA', 'FLAG_MASTERCARD', 'FLAG_DINERS',
       'FLAG_AMERICAN_EXPRESS', 'FLAG_OTHER_CARDS', 'QUANT_BANKING_ACCOUNTS',
       'QUANT_SPECIAL_BANKING_ACCOUNTS', 'PERSONAL_ASSETS_VALUE', 'QUANT_CARS',
       'COMPANY', 'PROFESSIONAL_STATE', 'PROFESSIONAL_CITY',
       'PROFESSIONAL_BOROUGH', 'FLAG_PROFESSIONAL_PHONE',
       'PROFESSIONAL_PHONE_AREA_CODE', 'MONTHS_IN_THE_JOB', 'PROFESSION_CODE',
       'OCCUPATION_TYPE', 'MATE_PROFESSION_CODE', 'MATE_EDUCATIO

### Functions for normalizing an delete columns

In [4]:
# function for normalizing data at once
def normalized_data(df):
    df_cop = df.copy()
    target_col = "TARGET_LABEL_BAD=1"

    # 'PAYMENT_DAY': category = ["1 - 15", "16 - 30"]
    df_cop["PAYMENT_DAY"] = np.where(df_cop["PAYMENT_DAY"] <= 14, "1_14", "15_30")

    # 'MARITAL_STATUS': category =  {1:'single', 2:'married', 3:'other'}
    df_cop["MARITAL_STATUS"] = np.where(
        df_cop["MARITAL_STATUS"] == 1,
        "single",
        np.where(df_cop["MARITAL_STATUS"] == 2, "married", "other"),
    )

    # 'QUANT_DEPENDANTS': numerical changes = [0, 1, 2, + 3]
    df_cop.loc[df_cop["QUANT_DEPENDANTS"] > 3, "QUANT_DEPENDANTS"] = 3
    # 'HAS_DEPENDANTS': categorical column = {0:False, >0:True}
    df_cop["HAS_DEPENDANTS"] = np.where(df_cop["QUANT_DEPENDANTS"] >= 1, True, False)
    df_cop["HAS_DEPENDANTS"] = df_cop["HAS_DEPENDANTS"].astype("bool")

    # "RESIDENCE_TYPE": numerical changes = {1: 'owned', 2:'mortgage', 3:'rented', 4:'family', 5:'other'}
    df_cop["RESIDENCE_TYPE"] = df_cop["RESIDENCE_TYPE"].fillna(0)

    # categorical changes
    # mapping = {1: "owned", 2: "mortgage", 3: "rented", 4: "family", 5: "other"}
    df_cop["HAS_RESIDENCE"] = np.where(df_cop["RESIDENCE_TYPE"] == 1, True, False)
    df_cop["HAS_RESIDENCE"] = df_cop["HAS_RESIDENCE"].astype("bool")

    # "MONTHS_IN_RESIDENCE": category = ['0 - 6 months', '< 1 year', '+ 1 year']
    df_cop["MONTHS_IN_RESIDENCE"] = np.where(
        df_cop["MONTHS_IN_RESIDENCE"] <= 6,
        "0_6",
        np.where(df_cop["MONTHS_IN_RESIDENCE"] <= 12, "6_12", ">_12"),
    )

    # "MONTHLY_INCOMES_TOT" and "OTHER_INCOMES" changed by "OTHER_INCOMES"
    # added to personal income in order to increase people who has less than minimal salary
    df_cop["MONTHLY_INCOMES_TOT"] = (
        df_cop["PERSONAL_MONTHLY_INCOME"] + df_cop["OTHER_INCOMES"]
    )

    df_cop["MONTHLY_INCOMES_TOT"] = pd.cut(
        df_cop["MONTHLY_INCOMES_TOT"],
        bins=[0, 650, 1320, 3323, 8560, float("inf")],
        labels=["[0_650]", "[650_1320]", "[1320_3323]", "[3323_8560]", "[>8560]"],
        right=False,
    )

    # 'HAS_CARDS' category, replaces all cards.
    list_cards = [
        "FLAG_VISA",
        "FLAG_MASTERCARD",
        "FLAG_DINERS",
        "FLAG_AMERICAN_EXPRESS",
        "FLAG_OTHER_CARDS",
    ]
    df_cop["HAS_CARDS"] = np.where(df_cop[list_cards].any(axis=1), True, False)
    df_cop["HAS_CARDS"] = df_cop["HAS_CARDS"].astype("bool")

    # "QUANT_BANKING_ACCOUNTS" and "QUANT_SPECIAL_BANKING_ACCOUNTS" changed by "HAS_BANKING_ACCOUNTS"
    # added to personal income in order to increase people who has less than minimal salary
    df_cop["QUANT_BANKING_ACCOUNTS"] = (
        df_cop["QUANT_BANKING_ACCOUNTS"] + df_cop["QUANT_SPECIAL_BANKING_ACCOUNTS"]
    )

    # 'HAS_BANKING_ACCOUNTS' category, replaces all accounts.
    df_cop["HAS_BANKING_ACCOUNTS"] = np.where(
        df_cop["QUANT_BANKING_ACCOUNTS"] == 0, False, True
    )
    df_cop["HAS_BANKING_ACCOUNTS"] = df_cop["HAS_BANKING_ACCOUNTS"].astype("bool")

    # 'PERSONAL_ASSETS_VALUE': changed to 'HAS_PERSONAL_ASSETS' = N, Y
    df_cop["HAS_PERSONAL_ASSETS"] = np.where(
        df_cop["PERSONAL_ASSETS_VALUE"] > 0, True, False
    )
    df_cop["HAS_PERSONAL_ASSETS"] = df_cop["HAS_PERSONAL_ASSETS"].astype("bool")

    # 'QUANT_CARS':changed to 'HAS_CARS' = N, Y
    df_cop["HAS_CARS"] = np.where(df_cop["QUANT_CARS"] == 0, False, True)
    df_cop["HAS_CARS"] = df_cop["HAS_CARS"].astype("bool")

    # "APPLICATION_SUBMISSION_TYPE": 0 values changed to "Carga"
    df_cop.loc[
        df_cop["APPLICATION_SUBMISSION_TYPE"] != "Web", "APPLICATION_SUBMISSION_TYPE"
    ] = "Carga"

    # 'SEX': deleted unknown values, changed to categorical
    df_cop.drop(
        df_cop[(df_cop["SEX"] == "N")].index,
        inplace=True,
    )
    df_cop.drop(
        df_cop[(df_cop["SEX"] == " ")].index,
        inplace=True,
    )

    # 'AGE'
    bins = [0, 18, 25, 35, 45, 60, float("inf")]
    labels = ["<_18", "18_25", "26_35", "36_45", "46_60", ">_60"]
    df_cop["AGE"] = pd.cut(df_cop["AGE"], bins=bins, labels=labels)

    return (df_cop, target_col)


def categorical_columns(df):
    # change columns to category, except bool columns
    object_columns = [col for col in df.columns if df[col].dtype != "bool"]
    df[object_columns] = df[object_columns].astype("category")
    return df


def delete_columns(df):
    # delete columns with single values
    num_unique_values = df.nunique()
    columns_to_drop = num_unique_values[num_unique_values == 1].index
    df.drop(columns=columns_to_drop, inplace=True)

    # delete columns according to our criteria
    drop_columns = [
        "ID_CLIENT",  # index
        "POSTAL_ADDRESS_TYPE",  # not valid proportion
        # 'QUANT_DEPENDANTS',  # delete??
        # 'HAS_DEPENDANTS', # delete??
        "STATE_OF_BIRTH",  # too many null values
        "CITY_OF_BIRTH",  # too many values
        "NACIONALITY",  # not valid proportion
        # RESIDENCIAL_STATE', # delete??
        "RESIDENCIAL_CITY",  # too many unique values
        "RESIDENCIAL_BOROUGH",  # too many unique values
        "RESIDENCIAL_PHONE_AREA_CODE",  # too many unique values
        # 'FLAG_RESIDENCIAL_PHONE', # DELETE? if not chart
        "RESIDENCE_TYPE",  # changed by HAS_RESIDENCE
        # "HAS_RESIDENCE", # DELETE?
        # "FLAG_EMAIL", # DELETE? if not chart
        "PERSONAL_MONTHLY_INCOME",  # changed by 'MONTHLY_INCOMES_TOT'
        "OTHER_INCOMES",  # changed by 'MONTHLY_INCOMES_TOT'
        "FLAG_VISA",  # replaced by 'HAS_CARDS'
        "FLAG_MASTERCARD",  # replaced by 'HAS_CARDS'
        "FLAG_DINERS",  # replaced by 'HAS_CARDS'
        "FLAG_AMERICAN_EXPRESS",  # replaced by 'HAS_CARDS'
        "FLAG_OTHER_CARDS",  # replaced by 'HAS_CARDS'
        "QUANT_BANKING_ACCOUNTS",  # replaced by 'HAS_BANKING_ACCOUNTS'
        "QUANT_SPECIAL_BANKING_ACCOUNTS",  # replaced by 'HAS_BANKING_ACCOUNTS'
        "PERSONAL_ASSETS_VALUE",  # replaced by 'HAS_PERSONAL_ASSETS'
        "QUANT_CARS",  # replaced by 'HAS_CARS'
        "PROFESSIONAL_STATE",  # more than 60% of empty values
        "PROFESSIONAL_CITY",  # too many different values
        "PROFESSIONAL_BOROUGH",  # too many different values
        "PROFESSIONAL_PHONE_AREA_CODE",  # more than 60% of empty values
        "MONTHS_IN_THE_JOB",  # more than 95% of 0 as a value
        "PROFESSION_CODE",  # not enough information, over 7k null values
        "OCCUPATION_TYPE",  # not enough information, over 7k null values
        "MATE_PROFESSION_CODE",  # over 50% of empty values
        "MATE_EDUCATION_LEVEL",  # over 60% of empty values
        # 'PRODUCT', delete?, 3 different values,
        "RESIDENCIAL_ZIP_3",  # too many unique values'
        "PROFESSIONAL_ZIP_3",
        
        'QUANT_ADDITIONAL_CARDS','EDUCATION_LEVEL', 
        'FLAG_MOBILE_PHONE', 'FLAG_HOME_ADDRESS_DOCUMENT', 
        'FLAG_RG', 'FLAG_CPF',
        'FLAG_INCOME_PROOF', 'PRODUCT', 'FLAG_ACSP_RECORD',
    ]  # too many unique values'

    list_not_find = []
    list_removed = []

    for outside_column in drop_columns:
        if outside_column in df.columns:
            list_removed.append(outside_column)
            df.drop(columns=outside_column, axis=1, inplace=True)
        else:
            list_not_find.append(outside_column)

    print("Those columns were removed: \n", list_removed)
    print("\nThose columns were not found: \n", list_not_find)

    return df

In [5]:
df_val_2, target = normalized_data(df_val)
df_val_2 = categorical_columns(df_val_2)
df_val_2 = delete_columns(df_val_2)

df_val_2.columns

Those columns were removed: 
 ['ID_CLIENT', 'POSTAL_ADDRESS_TYPE', 'STATE_OF_BIRTH', 'CITY_OF_BIRTH', 'NACIONALITY', 'RESIDENCIAL_CITY', 'RESIDENCIAL_BOROUGH', 'RESIDENCIAL_PHONE_AREA_CODE', 'RESIDENCE_TYPE', 'PERSONAL_MONTHLY_INCOME', 'OTHER_INCOMES', 'FLAG_VISA', 'FLAG_MASTERCARD', 'FLAG_DINERS', 'FLAG_AMERICAN_EXPRESS', 'FLAG_OTHER_CARDS', 'QUANT_BANKING_ACCOUNTS', 'QUANT_SPECIAL_BANKING_ACCOUNTS', 'PERSONAL_ASSETS_VALUE', 'QUANT_CARS', 'PROFESSIONAL_STATE', 'PROFESSIONAL_CITY', 'PROFESSIONAL_BOROUGH', 'PROFESSIONAL_PHONE_AREA_CODE', 'MONTHS_IN_THE_JOB', 'PROFESSION_CODE', 'OCCUPATION_TYPE', 'MATE_PROFESSION_CODE', 'MATE_EDUCATION_LEVEL', 'RESIDENCIAL_ZIP_3', 'PROFESSIONAL_ZIP_3', 'QUANT_ADDITIONAL_CARDS', 'EDUCATION_LEVEL', 'FLAG_MOBILE_PHONE', 'FLAG_HOME_ADDRESS_DOCUMENT', 'FLAG_RG', 'FLAG_CPF', 'FLAG_INCOME_PROOF', 'PRODUCT', 'FLAG_ACSP_RECORD']

Those columns were not found: 
 []


Index(['PAYMENT_DAY', 'APPLICATION_SUBMISSION_TYPE', 'SEX', 'MARITAL_STATUS',
       'QUANT_DEPENDANTS', 'RESIDENCIAL_STATE', 'FLAG_RESIDENCIAL_PHONE',
       'MONTHS_IN_RESIDENCE', 'FLAG_EMAIL', 'COMPANY',
       'FLAG_PROFESSIONAL_PHONE', 'AGE', 'HAS_DEPENDANTS', 'HAS_RESIDENCE',
       'MONTHLY_INCOMES_TOT', 'HAS_CARDS', 'HAS_BANKING_ACCOUNTS',
       'HAS_PERSONAL_ASSETS', 'HAS_CARS'],
      dtype='object')

### Encode columns

### Load Model and list columns

In [6]:
reg_model = pickle.load(open('logistic_regression.pk', 'rb')) # to load model...
print(type(reg_model))
print(reg_model.feature_names_in_)

<class 'sklearn.model_selection._search.GridSearchCV'>
['PAYMENT_DAY_1_14' 'APPLICATION_SUBMISSION_TYPE_Web' 'SEX_M'
 'MARITAL_STATUS_other' 'MARITAL_STATUS_single' 'QUANT_DEPENDANTS_1'
 'QUANT_DEPENDANTS_2' 'QUANT_DEPENDANTS_3' 'RESIDENCIAL_STATE_AL'
 'RESIDENCIAL_STATE_AM' 'RESIDENCIAL_STATE_AP' 'RESIDENCIAL_STATE_BA'
 'RESIDENCIAL_STATE_CE' 'RESIDENCIAL_STATE_DF' 'RESIDENCIAL_STATE_ES'
 'RESIDENCIAL_STATE_GO' 'RESIDENCIAL_STATE_MA' 'RESIDENCIAL_STATE_MG'
 'RESIDENCIAL_STATE_MS' 'RESIDENCIAL_STATE_MT' 'RESIDENCIAL_STATE_PA'
 'RESIDENCIAL_STATE_PB' 'RESIDENCIAL_STATE_PE' 'RESIDENCIAL_STATE_PI'
 'RESIDENCIAL_STATE_PR' 'RESIDENCIAL_STATE_RJ' 'RESIDENCIAL_STATE_RN'
 'RESIDENCIAL_STATE_RO' 'RESIDENCIAL_STATE_RR' 'RESIDENCIAL_STATE_RS'
 'RESIDENCIAL_STATE_SC' 'RESIDENCIAL_STATE_SE' 'RESIDENCIAL_STATE_SP'
 'RESIDENCIAL_STATE_TO' 'FLAG_RESIDENCIAL_PHONE_Y'
 'MONTHS_IN_RESIDENCE_6_12' 'MONTHS_IN_RESIDENCE_>_12' 'FLAG_EMAIL_1'
 'COMPANY_Y' 'FLAG_PROFESSIONAL_PHONE_Y' 'PRODUCT_2' 'PRODUCT_7'
 '

### Encode dataset, fixed name and arrange columns following the model

In [7]:
df_encoded = pd.get_dummies(data=df_val_2, drop_first=True)

mapeo_nombres = {'HAS_CARDS': 'HAS_CARDS_True', 'HAS_CARS': 'HAS_CARS_True', 'HAS_DEPENDANTS': 'HAS_DEPENDANTS_True',
                'HAS_BANKING_ACCOUNTS': 'HAS_BANKING_ACCOUNTS_True', 'AGE_18_25':'AGE_<_18', 
                'HAS_PERSONAL_ASSETS':'HAS_PERSONAL_ASSETS_True', 'HAS_RESIDENCE':'HAS_RESIDENCE_True'}

df_encoded.rename(columns=mapeo_nombres, inplace=True)

# assume the product is equal to 1
df_encoded['PRODUCT_2'] = 0
df_encoded['PRODUCT_7'] = 0
df_encoded['PRODUCT_2'] = df_encoded['PRODUCT_2'].astype('uint8')
df_encoded['PRODUCT_7'] = df_encoded['PRODUCT_7'].astype('uint8')


In [8]:
df_reg = df_encoded[['PAYMENT_DAY_1_14', 'APPLICATION_SUBMISSION_TYPE_Web', 'SEX_M',
       'MARITAL_STATUS_other', 'MARITAL_STATUS_single', 'QUANT_DEPENDANTS_1',
       'QUANT_DEPENDANTS_2', 'QUANT_DEPENDANTS_3', 'RESIDENCIAL_STATE_AL',
       'RESIDENCIAL_STATE_AM', 'RESIDENCIAL_STATE_AP', 'RESIDENCIAL_STATE_BA',
       'RESIDENCIAL_STATE_CE', 'RESIDENCIAL_STATE_DF', 'RESIDENCIAL_STATE_ES',
       'RESIDENCIAL_STATE_GO', 'RESIDENCIAL_STATE_MA', 'RESIDENCIAL_STATE_MG',
       'RESIDENCIAL_STATE_MS', 'RESIDENCIAL_STATE_MT', 'RESIDENCIAL_STATE_PA',
       'RESIDENCIAL_STATE_PB', 'RESIDENCIAL_STATE_PE', 'RESIDENCIAL_STATE_PI',
       'RESIDENCIAL_STATE_PR', 'RESIDENCIAL_STATE_RJ', 'RESIDENCIAL_STATE_RN',
       'RESIDENCIAL_STATE_RO', 'RESIDENCIAL_STATE_RR', 'RESIDENCIAL_STATE_RS',
       'RESIDENCIAL_STATE_SC', 'RESIDENCIAL_STATE_SE', 'RESIDENCIAL_STATE_SP',
       'RESIDENCIAL_STATE_TO', 'FLAG_RESIDENCIAL_PHONE_Y',
       'MONTHS_IN_RESIDENCE_6_12', 'MONTHS_IN_RESIDENCE_>_12', 'FLAG_EMAIL_1',
       'COMPANY_Y', 'FLAG_PROFESSIONAL_PHONE_Y', 'PRODUCT_2', 'PRODUCT_7',
       'AGE_26_35', 'AGE_36_45', 'AGE_46_60', 'AGE_<_18', 'AGE_>_60',
       'HAS_DEPENDANTS_True', 'HAS_RESIDENCE_True',
       'MONTHLY_INCOMES_TOT_[1320_3323]', 'MONTHLY_INCOMES_TOT_[3323_8560]',
       'MONTHLY_INCOMES_TOT_[650_1320]', 'MONTHLY_INCOMES_TOT_[>8560]',
       'HAS_CARDS_True', 'HAS_BANKING_ACCOUNTS_True',
       'HAS_PERSONAL_ASSETS_True', 'HAS_CARS_True']]

### Run the model on testing dataset

In [9]:
val = reg_model.predict(df_reg)


### Add val to main df

In [10]:
df_reg['TARGET_LABEL_BAD=1'] = val

C:\Users\Yessid\AppData\Local\Temp\ipykernel_15632\2586380496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reg['TARGET_LABEL_BAD=1'] = val


In [11]:
df_reg.head()

,PAYMENT_DAY_1_14,APPLICATION_SUBMISSION_TYPE_Web,SEX_M,MARITAL_STATUS_other,MARITAL_STATUS_single,QUANT_DEPENDANTS_1,QUANT_DEPENDANTS_2,QUANT_DEPENDANTS_3,RESIDENCIAL_STATE_AL,RESIDENCIAL_STATE_AM,...,HAS_RESIDENCE_True,MONTHLY_INCOMES_TOT_[1320_3323],MONTHLY_INCOMES_TOT_[3323_8560],MONTHLY_INCOMES_TOT_[650_1320],MONTHLY_INCOMES_TOT_[>8560],HAS_CARDS_True,HAS_BANKING_ACCOUNTS_True,HAS_PERSONAL_ASSETS_True,HAS_CARS_True,TARGET_LABEL_BAD=1
0,1,1,0,0,1,0,0,0,0,0,...,True,0,0,1,0,True,True,False,True,1
1,1,1,0,0,1,0,0,0,0,0,...,True,0,0,0,0,False,False,False,False,0
2,0,1,0,0,0,0,0,0,0,0,...,True,0,0,1,0,False,False,False,False,0
3,1,1,0,0,1,0,0,0,0,0,...,True,0,0,0,0,False,False,False,False,0
4,0,1,0,0,1,0,0,0,0,0,...,True,0,0,1,0,False,False,False,False,1


### Saving the df as csv file

In [12]:
name = 'app_test_prediction.csv'
df_reg.to_csv(name, index=False)